In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from random import shuffle
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier,LogisticRegression

path = '/content/drive/MyDrive/data_23/'

groups = {'general': ['MentalHealthSupport','mentalhealth','mental','personalitydisorders',
                      'mentalillness','MentalHealthPH'],
          
          'control': ['askscience','askscience2','LifeProTips','LifeProTips2','AskReddit','AskReddit2',
                      'answers','answers2', 'AskScienceFiction','AskScienceFiction2','TrueAskReddit',
                      'TrueAskReddit2'],

          'adhd': ['ADHD','ADHD2'],

          'autism': ['aspergers','autism2','AutisticQueers','AutismInWomen','Aspergers_Elders',
                     'asperger','AutisticPride','autism','AutismTranslated','aspergers_dating',
                     'aspergirls','AutisticAdults'],
          
          'anxiety': ['anxiety'],

          'ocd': ['OCD'],

          'ptsd': ['ptsd','CPTSD'],

          'phobia': ['Phobia','emetophobia','Agoraphobia'],

          'socialanxiety':['socialanxiety','socialanxiety2'],
          
          'depression': ['depression1','depression2','depression3'],

          'sadness': ['sad11','sad22','sad33'],
          
          'bipolar': ['bipolar','BipolarReddit'],
          
          'schizophrenia': ['schizophrenia','paranoidschizophrenia','schizoaffective','Psychosis'],

          'cluster_a': ['Schizoid','Schizotypal','ParanoidPersonality',
                        'Paranoid','ParanoiaCheck','Paranoia'],
          'cluster_b': ['BorderlinePDisorder','BPD','Borderline','hpd','NPD','narcissism',
                        'sociopath', 'psychopath','Psychopathy','aspd'],
          'cluster_c': ['OCPD','AvPD','Avoidant', 'DPD'],

          'selfharm': ['selfharm','StopSelfHarm','AdultSelfHarm',
                       'SuicideWatch11','SuicideWatch22','SuicideWatch33'],
          
          'addiction': ['addiction','alcoholism'],

          'eating': ['ARFID', 'bulimia','eating_disorders','EDAnonymous','EatingDisorders'],

          'dpdr': ['dpdr'],
          'dysmorphic': ['DysmorphicDisorder', 'BodyAcceptance'],
          'tourettes': ['Tourettes'],
          'anger': ['Anger'],
          }


In [ ]:
os.chdir(path)
i=1
lst=os.listdir()
shuffle(lst)

hash = HashingVectorizer(ngram_range=(1, 2), stop_words='english', alternate_sign=False)
tfi = TfidfTransformer()
clf = MultinomialNB(alpha=.01)

df = pd.DataFrame(columns=['x','y'])
for f in lst[:20]:
    df_ = pd.read_csv(f, lineterminator='\n', engine='c')
    df_ = df_[[len(i)>500 for i in df_.x]]
    df = pd.concat([df,df_], ignore_index=True)
y_test = df.y.values
x_test = hash.fit_transform(df.x.values)
x_test = tfi.fit_transform(x_test)
del df, df_

for i,f in enumerate(lst[20:]):
    df = pd.read_csv(f, lineterminator='\n', engine='c')
    df = df[[len(i)>500 for i in df.x]]
    x_train = hash.fit_transform(df.x.values)
    x_train = tfi.fit_transform(x_train)

    clf.partial_fit(x_train, df.y.values, classes=list(groups.keys()))
    s = clf.score(x_test, y_test)
    print(i, ', score:', s)
    if not i%1000:
        pickle.dump(clf,open('/content/drive/MyDrive/model/MNB_stop_Bi_Tfi_N'+str(i)+'_L500_'+str(round(s,4))+'.pkl', 'wb'))
    
pickle.dump(clf,open('/content/drive/MyDrive/model/MNB_stop_Bi_Tfi_N'+str(i)+'_L500_'+str(round(s,4))+'.pkl', 'wb'))

Streaming output truncated to the last 5000 lines.
4614 , score: 0.21280738636667576
4615 , score: 0.21281808504378433
4616 , score: 0.21284483173655577
4617 , score: 0.21277529033535003
4618 , score: 0.21281273570523004
4619 , score: 0.21277529033535003
4620 , score: 0.21271109827269857
4621 , score: 0.21271644761125286
4622 , score: 0.2127913383510129
4623 , score: 0.21282878372089292
4624 , score: 0.21286087975221865
4625 , score: 0.21290902379920723
4626 , score: 0.21293577049197868
4627 , score: 0.2129304211534244
4628 , score: 0.21291437313776151
4629 , score: 0.21290902379920723
4630 , score: 0.21292507181487008
4631 , score: 0.2129197224763158
4632 , score: 0.2129197224763158
4633 , score: 0.21292507181487008
4634 , score: 0.21289832512209866
4635 , score: 0.2128234343823386
4636 , score: 0.21280738636667576
4637 , score: 0.21280203702812148
4638 , score: 0.2127431943040243
4639 , score: 0.21272714628836145
4640 , score: 0.21270039959559
4641 , score: 0.21271109827269857
4642 ,

In [ ]:
os.chdir(path)
lst=os.listdir()
shuffle(lst)
df = pd.DataFrame(columns=['x'])

for i,f in enumerate(lst[:1200]):
    print(i, end=' ')
    if i>5 and not i%25:
        print('')
    df_ = pd.read_csv(f, usecols=['x'], lineterminator='\n', engine='c')
    df_ = df_[[len(i)>500 for i in df_.x]]
    df = pd.concat([df,df_], ignore_index=True)

hash_tfi = Pipeline([('hash', HashingVectorizer(ngram_range=(1, 2), 
                                                stop_words='english', 
                                                alternate_sign=False)),
                     ('tfi', TfidfTransformer())])
hash_tfi.fit(df.x.values)
pickle.dump(hash_tfi,open('/content/drive/MyDrive/model/hash_tfi_1200.pkl', 'wb'))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 
26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 
76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 
126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 
151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 
176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 
226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 
251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import gzip
import pickle

!pip install keras-tuner
import kerastuner as kt

from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Flatten, Maximum, MaxPooling2D, Conv2D, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

train_images = np.load(gzip.open('/content/drive/MyDrive/train_images.npy.gz', 'rb'))
train_labels = np.load(gzip.open('/content/drive/MyDrive/train_labels.npy.gz', 'rb'))
validation_images = np.load(gzip.open('/content/drive/MyDrive/validation_images.npy.gz', 'rb'))

pretrained = pickle.load(open('/content/drive/MyDrive/pretrained.pkl', 'rb'))
pre_valid = pickle.load(open('/content/drive/MyDrive/pre_valid.pkl', 'rb'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.9 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  if __name__ == '__main__':


In [ ]:
def build_model(hp):
    model = keras.Sequential()
    n = hp.Choice('n', [2, 3, 4,5,6,7,8,9,10,11,12,13])
    for i in range(n):
        model.add(Dense(hp.Int(f'units_{i}', min_value=30, max_value=900, step=20), 
                        activation=hp.Choice(f'act_{i}',['relu','elu','leaky_relu'])))
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(rate=hp.Float(f'drop_rate_{i}', min_value=.15, max_value=.25)))
    model.add(Dense(10, activation='softmax'))
    model.build(input_shape=(50000, 2048))
    model.compile(loss='CategoricalCrossentropy', 
                  optimizer=keras.optimizers.Adam(learning_rate=hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")), 
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    max_trials=200,
    objective='val_accuracy')

with tf.device('/device:GPU:0'):
    tuner.search(pretrained, tf.one_hot(train_labels,depth=10), 
                 epochs=40, batch_size=64, verbose=1, validation_split=0.2)

Trial 200 Complete [00h 01m 28s]
val_accuracy: 0.8554999828338623

Best val_accuracy So Far: 0.8633000254631042
Total elapsed time: 05h 36m 40s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tuner.get_best_models()[0]
predictions = model.predict(pre_valid)
predicted_classes = np.argmax(predictions, axis=1)
pd.DataFrame(predicted_classes).to_csv('aa.csv')

In [ ]:
tuner.get_best_hyperparameters(1)[0].values

{'act_0': 'leaky_relu',
 'act_1': 'relu',
 'act_10': 'elu',
 'act_11': 'leaky_relu',
 'act_12': 'relu',
 'act_2': 'relu',
 'act_3': 'relu',
 'act_4': 'leaky_relu',
 'act_5': 'leaky_relu',
 'act_6': 'leaky_relu',
 'act_7': 'relu',
 'act_8': 'leaky_relu',
 'act_9': 'relu',
 'drop_rate_0': 0.22335689534040387,
 'drop_rate_1': 0.15370697296646973,
 'drop_rate_10': 0.19579885544749498,
 'drop_rate_11': 0.15409387439486796,
 'drop_rate_12': 0.2089962251512538,
 'drop_rate_2': 0.2074783443989959,
 'drop_rate_3': 0.22441652123588246,
 'drop_rate_4': 0.1894328644276993,
 'drop_rate_5': 0.22902797574515887,
 'drop_rate_6': 0.22395892596362782,
 'drop_rate_7': 0.23231202626059463,
 'drop_rate_8': 0.2416056590420367,
 'drop_rate_9': 0.17074264239351225,
 'dropout_0': False,
 'dropout_1': False,
 'dropout_10': True,
 'dropout_11': True,
 'dropout_12': False,
 'dropout_2': False,
 'dropout_3': False,
 'dropout_4': True,
 'dropout_5': False,
 'dropout_6': True,
 'dropout_7': False,
 'dropout_8': True

In [ ]:
Trial 74 Complete [00h 01m 37s]
val_accuracy: 0.857699990272522

Best val_accuracy So Far: 0.8633000254631042
Total elapsed time: 02h 04m 19s

Search: Running Trial #75

Value             |Best Value So Far |Hyperparameter
6                 |3                 |n
550               |470               |units_0
relu              |leaky_relu        |act_0
True              |False             |dropout_0
630               |770               |units_1
relu              |relu              |act_1
True              |False             |dropout_1
0.0025572         |0.00014019        |lr
0.23393           |0.15371           |drop_rate_1
630               |390               |units_2
leaky_relu        |relu              |act_2
True              |False             |dropout_2
0.23406           |0.22336           |drop_rate_0
370               |190               |units_3
elu               |relu              |act_3
False             |False             |dropout_3
730               |710               |units_4
relu              |leaky_relu        |act_4
False             |True              |dropout_4
750               |390               |units_5
relu              |leaky_relu        |act_5
True              |False             |dropout_5
550               |170               |units_6
relu              |leaky_relu        |act_6
True              |True              |dropout_6
610               |750               |units_7
elu               |relu              |act_7
False             |False             |dropout_7
610               |190               |units_8
relu              |leaky_relu        |act_8
True              |True              |dropout_8
650               |630               |units_9
leaky_relu        |relu              |act_9
False             |False             |dropout_9
570               |710               |units_10
leaky_relu        |elu               |act_10
True              |True              |dropout_10
0.24429           |0.20748           |drop_rate_2
0.21083           |0.22442           |drop_rate_3
0.17912           |0.18943           |drop_rate_4
0.24792           |0.22396           |drop_rate_6
0.20913           |0.23231           |drop_rate_7
0.2171            |0.17074           |drop_rate_9
0.15787           |0.22903           |drop_rate_5
0.17677           |0.24161           |drop_rate_8
310               |370               |units_11
elu               |leaky_relu        |act_11
True              |True              |dropout_11
130               |730               |units_12
elu               |relu              |act_12
True              |False             |dropout_12
0.20654           |0.15409           |drop_rate_11
0.22483           |0.209             |drop_rate_12
0.19861           |0.1958            |drop_rate_10

